In [1]:
import re
import jsonschema
from jsonschema import TypeChecker
import json
from tqdm import tqdm
import os
import xmltodict, json

In [29]:
def extract_text_between_sample_tags(text, sample_tag=False):
    # Define the regular expression pattern to match text between [SAMPLE] and [/SAMPLE]
    if sample_tag:
        pattern = r'\[SAMPLE\]\n(.*?)\[/SAMPLE\]'
    else:
        # pattern = r'```XML sample:(.*?)```'
        # pattern = r'Answer:\n```json\n(.*?)```'
        # pattern = r'''JSON sample:\n```json\n(.*?)```'''
        # pattern = r'''JSON sample:\n```(.*?)```'''
        pattern = r'''```(.*?)```'''

    
    # Use re.findall to find all matches
    matches = re.findall(pattern, text, re.DOTALL)
    
    if len(matches)==0 and sample_tag==False:
        # pattern = r'''Answer:\n```(.*?)```'''
        # pattern = r'''JSON sample:\n```(.*?)```'''
        pattern = r'''```(.*?)```'''
        # pattern = r'```json(.*?)```'

        matches = re.findall(pattern, text, re.DOTALL)
        
    return matches

In [3]:
def get_paths(d, prefix='schema'):
    paths = []
    for key, value in d.items():
        if prefix:
            new_key = f'''{prefix}['{key}']'''
        else:
            new_key = key
        paths.append(new_key)
        if isinstance(value, dict):
            paths.extend(get_paths(value, new_key))
    return paths

In [19]:
# output_base_path = "/raid/nlp/sameer/conCodeEval/all_data/dccstor/ai4code-summ/benchmark-paper/backup/constrain-data-gen-eval/data_generator/data_generator/Task_1/x_to_JSON/JSON_2_JSON/granite/34B/zero_shot/prompt_1/"
output_base_path = "/raid/nlp/sameer/conCodeEval/all_data/dccstor/ai4code-summ/benchmark-paper/backup/constrain-data-gen-eval/data_generator/data_generator/Task_1/x_to_JSON/JSON_2_JSON/codellama/70B/zero_shot/prompt_2/"

output_json_files = os.listdir(output_base_path)

base_path = "/raid/nlp/sameer/conCodeEval/all_data/dccstor/ai4code-summ/benchmark-paper/backup/constrain-data-gen-eval/data_generator/data_generator/"
json_schema_path = "json_schema_dataset_28_feb/"
xml_schema_path = "xml_dataset_6_march/"
yaml_schema_path = "yaml_dataset_6_march/"
manual_NL_path = "manual_NL_summary/"

In [5]:
errors = []
invalid_json = 0
incomplete_json = 0
type_violation = 0
file_name_errors = {}
incomplete_files = []

In [6]:
# This function divides the schema constraints in starting 20%, end 20%, and rest middle
def bucketing_constraints(all_keys_string):
    begin = len(all_keys_string)/5
    end = len(all_keys_string) - len(all_keys_string)/5
    begin_constraints = []
    mid_constraints = []
    end_constraints = []
    for i in range(0,len(all_keys_string)):
        if i < begin:
            begin_constraints.append(all_keys_string[i])
        elif i >= end:
            end_constraints.append(all_keys_string[i])
        else:
            mid_constraints.append(all_keys_string[i])

    return begin_constraints, mid_constraints, end_constraints

In [35]:
invalid_json, incomplete_json = 0, 0

incomplete_files = []
total_begin_errors, total_mid_errors, total_end_errors = [], [], []

for file in tqdm(output_json_files):
    if file.endswith(".txt") == False:
        continue
        
    f_output = open(output_base_path+file, "r")
    text = f_output.read().split('<|eot_id|><|start_header_id|>assistant<|end_header_id|>')[1]
    f_schema = open(base_path+json_schema_path+file.split(".")[0] + ".json", "r")
    schema_text = f_schema.read()
    schema = json.loads(schema_text)
    all_keys_string = get_paths(schema)
    begin_constraints, mid_constraints, end_constraints = bucketing_constraints(all_keys_string)
    extracted_texts = extract_text_between_sample_tags(text.strip(), sample_tag=False)
    if len(extracted_texts) == 0:
        incomplete_files.append(file)
        incomplete_json += 1
    else:
        for i, extracted_text in enumerate(extracted_texts, 1):
            try:
                json_sample = extracted_text.strip()
                if json_sample.startswith(".json"):
                    json_sample = "\n".join(json_sample.split("\n")[1:])
#                 json_sample = json.dumps(yaml.safe_load(json_sample), default=str)
                # json_sample = json.dumps(xmltodict.parse(json_sample), default=str)
#                 o = xmltodict.parse('<e> <a>text</a> <a>text</a> </e>')
                gen_json = json.loads(json_sample)
                validator = jsonschema.Draft7Validator(schema)
                i = 0
                begin_errors, mid_errors, end_errors = 0, 0, 0
                for error in validator.iter_errors(gen_json):
                    error_str = str(error)
                    error_tokens = error_str.split("\n\n")[1].split()
                    actual_error = error_tokens[4][:-1]+"["+error_tokens[2]+"]"

                    if actual_error in begin_constraints:
                        begin_errors += 1
                    if actual_error in mid_constraints:
                        mid_errors += 1
                    if actual_error in end_constraints:
                        end_errors += 1

                total_begin_errors.append(begin_errors)
                total_end_errors.append(end_errors)
                total_mid_errors.append(mid_errors)
                errors.append(i)
            except:
                invalid_json += 1
            break


100%|██████████| 602/602 [00:00<00:00, 4537.99it/s]


In [16]:
sum(total_begin_errors), sum(total_mid_errors), sum(total_end_errors) #granite 8b

(199, 160, 57)